<a href="https://colab.research.google.com/github/RamsesMDLC/Smolagent_Project_1/blob/main/Smolagents_Project_1_YT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
x = 1
y = 9
z = x + y
print(z)

10


In [3]:
!pip install smolagents[toolkit]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 145.4/145.4 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.3/5.3 MB 58.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 62.9 MB/s eta 0:00:00
  Attempting uninstall: lxml
    Found existing installation: lxml 5.4.0
    Uninstalling lxml-5.4.0:
      Successfully uninstalled lxml-5.4.0


In [4]:
from google.colab import userdata
from huggingface_hub import login

hf_token = userdata.get('HF_TOKEN')  # Retrieves the token securely
if hf_token:
    login(hf_token)
    print("Successfully logged in to Hugging Face!")
else:
    print("Token not found. Please add HF_TOKEN secret.")


Successfully logged in to Hugging Face!


In [5]:
#transformersModel:
  #model abstraction or wrapper that uses Hugging Face's Transformers library to load, manage, and run SOTA Transformer-based models.
  #Through "transformersModel", the Transformer-based LLMs are accessible and usable in frameworks like smolagents for tasks involving chat,...
  #...code generation, search
  #It abstracts away the complexity of tokenization, inference, and managing model pipelines, allowing agent frameworks (like smolagents)...
  #...to use powerful language models with simple APIs.

from smolagents import CodeAgent, DuckDuckGoSearchTool, TransformersModel

In [20]:
model = TransformersModel(model_id="Qwen1.5-1.8B")
agent = CodeAgent(tools=[DuckDuckGoSearchTool()], model=model, max_new_tokens=4096)

ValueError: Failed to load tokenizer and model for model_id='Qwen1.5-1.8B': Qwen1.5-1.8B is not a local folder and is not a valid model identifier listed on 'https://huggingface.co/models'
If this is a private repository, make sure to pass a token having permission to this repo either by logging in with `hf auth login` or by passing `token=<your_token>`

In [ ]:
agent.run("How long would it take for an elephant to cross the United States from Florida to California")

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ How long would it take for an elephant to cross the United States from Florida to California                    │
│                                                                                                                 │
╰─ TransformersModel - HuggingFaceTB/SmolLM2-1.7B ────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Error in generating model output:
Cannot use chat template functions because tokenizer.chat_template is not set and no template argument was passed! 
For information about writing templates and setting the tokenizer.chat_template attribute, please see the 
documentation at https://huggingface.co/docs/transformers/main/en/chat_templating

[Step 1: Duration 0.00 seconds]

AgentGenerationError: Error in generating model output:
Cannot use chat template functions because tokenizer.chat_template is not set and no template argument was passed! For information about writing templates and setting the tokenizer.chat_template attribute, please see the documentation at https://huggingface.co/docs/transformers/main/en/chat_templating